# A Simple ML approach to race result prediction in F1
## using historical and modern F1 data to dynamically adjust the model over time 

Introduction
Formula 1 is a data-driven sport where the outcome of a race depends on a combination of driver skill, team strategy, car performance, and external factors such as weather and track characteristics. With access to rich datasets, such as telemetry, lap times, and historical results, it is possible to leverage machine learning techniques to predict race outcomes. This project aims to explore and model the complex interplay of these factors to gain insights into what influences driver performance and race results.

Project Goal
The goal of this project is to develop a machine learning model capable of predicting Formula 1 race outcomes, specifically the finishing positions of drivers, based on:

- Qualifying session data (grid position, sector times, tire compounds).
- Historical performance of drivers and teams at specific tracks.
- Weather conditions and track-specific characteristics.
- Team-level metrics, such as reliability and average performance.
This model will provide a data-driven approach to understanding and predicting race results, offering valuable insights for analysts, fans, and potentially teams themselves.

In [3]:
#!pip install fastf1 pandas numpy matplotlib seaborn scikit-learn xgboost


Lets start by importing the libraries used for the data collection process.

In [5]:
import fastf1

fastf1.Cache.enable_cache('../data/cache/')

In [6]:
years = range(2018,2025)
sessionids = {year: list(
        range(1, len(fastf1.get_event_schedule(year)[
            fastf1.get_event_schedule(year).EventFormat != 'testing'
        ])+1)
    )
              for year in years}

In [7]:
session_data = {
    'Practice': [],
    'Qualifying': [],
    'Race': []
}

In [8]:
def collect_race_weekend_dataFF1(year, event_id):
    # Define raceID
    race_id = f"{year}_{event_id}"
    
    # Load the event
    event = fastf1.get_event(year, event_id)
    
    # Iterate over all sessions in the weekend
    for sessionID in range(1,6):
        try:
            session = event.get_session(sessionID)
            session_name = session.name
            session.load()
            print(f"Loaded session: {sessionID} for Race {race_id}")
            
            # Extract laps and results, add raceID
            laps = session.laps.copy()
            results = session.results.copy()
            
            laps['raceID'] = race_id
            results['raceID'] = race_id
            
            # Append data to appropriate session type
            if 'Practice' in session_name:
                session_data['Practice'].append(laps)
            elif session_name == 'Qualifying':
                session_data['Qualifying'].append(laps)
            elif session_name == 'Race':
                session_data['Race'].append(results)
        
        except Exception as e:
            print(f"Error loading {session_name} for Race {race_id}: {e}")

In [9]:
# Main loop to iterate through the dictionary
for year, event_ids in sessionids.items():
    if year >= 2024:
       for event_id in event_ids:
                #print(event_id)
                collect_race_weekend_dataFF1(year, event_id)

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2024/1/results.json failed; using cached response
Traceback (most recent call last):
  File "C:\Users\vnvtr\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\vnvtr\anaconda3\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vnvtr\anaconda3\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\vnvtr\anaconda3\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vnvtr\anaconda3\Lib\http\cli

Loaded session: 1 for Race 2024_1


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 3 [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for tra

Loaded session: 2 for Race 2024_1


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

Loaded session: 3 for Race 2024_24
Error loading Practice 3 for Race 2024_24: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger      WARNING 	Failed to load timing data!
req            INFO

Loaded session: 4 for Race 2024_24
Error loading Qualifying for Race 2024_24: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No 

Loaded session: 5 for Race 2024_24
Error loading Race for Race 2024_24: The data you are trying to access has not been loaded yet. See `Session.load`


In [10]:
import pandas as pd
import os 

for session_type, data_list in session_data.items():
    combined_df = pd.concat(data_list, ignore_index=True)
    output_path = os.path.join("..", "data", f"{session_type}_data.csv")
    # Determine write mode based on file existence
    file_exists = os.path.exists(output_path)

    if file_exists: 
        combined_df.to_csv(
            output_path,
            mode='a',  # Append mode
            header=not file_exists,  # Write header only if the file does not exist
            index=False
        )
    else: 
        combined_df.to_csv(output_path, index=False)
        print(f"Saved {session_type} data with {len(combined_df)} rows.")

In [11]:
fast

NameError: name 'fast' is not defined

In [ ]:
fastf1.get_event_schedule(2022)